In [4]:
# Let's write a more extensive example that uses Closing Price and TakerBuyVolume as features for an SVM model.
# This example assumes you have a CSV file with your data as described.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Load your data from CSV
file_path = '../Dataframes/1-Combined_Data'    
df = pd.read_csv("1-Combined_Data.csv")

# Feature Engineering
# For this example, we'll just use Closing Price and TakerBuyVolume directly
X = df[['ClosePrice', 'TakerBuyVolume']]

# Labeling - Creating a simplistic label for demonstration
# This is a placeholder. You need to replace it with your logic for Buy, Sell, Hold
# For example, comparing close price of the next row (future price) with the current row (present price)
# Here, shifting the ClosePrice column up to compare the next period's price with the current period's price
df['FutureClosePrice'] = df['ClosePrice'].shift(-1)
df['Action'] = 'Hold'  # Default to 'Hold'
df.loc[df['FutureClosePrice'] > df['ClosePrice'] * 1.01, 'Action'] = 'Buy'  # If the future price is >1% higher, 'Buy'
df.loc[df['FutureClosePrice'] < df['ClosePrice'] * 0.99, 'Action'] = 'Sell'  # If the future price is >1% lower, 'Sell'

y = df['Action'][:-1]  # Remove the last row since it has no future price to compare with
X = X[:-1]  # Ensure X and y have the same length

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM Model
model = SVC(kernel='rbf')
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Note: The labeling logic above is very simplistic and should be adjusted according to your strategy and dataset specifics.